In [1]:
spark.sparkContext.getConf().get('spark.driver.memory')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1606841923384_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'1000M'

In [2]:
%%configure -f 
{"driverMemory": "8000M"}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1606841923384_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1606841923384_0006,pyspark,idle,Link,Link,✔


In [4]:
from pyspark.sql import SQLContext
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.us-east-1.amazonaws.com")
sqlContext = SQLContext(sc)
match = sqlContext.read.format('com.databricks.spark.csv').option('delimiter', '\t').load('s3://millionsongprocessed/train_triplets.txt')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
spark.sparkContext.getConf().get('spark.driver.memory')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'8000M'

In [6]:
match.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+---+
|                 _c0|               _c1|_c2|
+--------------------+------------------+---+
|b80344d063b5ccb32...|SOAKIMP12A8C130995|  1|
|b80344d063b5ccb32...|SOAPDEY12A81C210A9|  1|
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|  2|
|b80344d063b5ccb32...|SOBFNSP12AF72A0E22|  1|
|b80344d063b5ccb32...|SOBFOVM12A58A7D494|  1|
|b80344d063b5ccb32...|SOBNZDC12A6D4FC103|  1|
|b80344d063b5ccb32...|SOBSUJE12A6D4F8CF5|  2|
|b80344d063b5ccb32...|SOBVFZR12A6D4F8AE3|  1|
|b80344d063b5ccb32...|SOBXALG12A8C13C108|  1|
|b80344d063b5ccb32...|SOBXHDL12A81C204C0|  1|
+--------------------+------------------+---+
only showing top 10 rows

In [7]:
match = match.rdd.map(lambda p: (p[0], p[1], int(p[2])))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
from pyspark.sql.types import IntegerType, StringType, StructField, StructType
schemaTaste = StructType([StructField("user", StringType(), True), StructField("song", StringType(), True), StructField("playCount", IntegerType(), True)])
dfTaste = sqlContext.createDataFrame(match, schema=schemaTaste)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
dfTaste.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+---------+
|                user|              song|playCount|
+--------------------+------------------+---------+
|b80344d063b5ccb32...|SOAKIMP12A8C130995|        1|
|b80344d063b5ccb32...|SOAPDEY12A81C210A9|        1|
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|        2|
|b80344d063b5ccb32...|SOBFNSP12AF72A0E22|        1|
|b80344d063b5ccb32...|SOBFOVM12A58A7D494|        1|
|b80344d063b5ccb32...|SOBNZDC12A6D4FC103|        1|
|b80344d063b5ccb32...|SOBSUJE12A6D4F8CF5|        2|
|b80344d063b5ccb32...|SOBVFZR12A6D4F8AE3|        1|
|b80344d063b5ccb32...|SOBXALG12A8C13C108|        1|
|b80344d063b5ccb32...|SOBXHDL12A81C204C0|        1|
+--------------------+------------------+---------+
only showing top 10 rows

In [10]:
from pyspark.sql import functions as F
# change ids from strings to integers
user_change = dfTaste.select('user').distinct().select('user', F.monotonically_increasing_id().alias('new_user'))
song_change = dfTaste.select('song').distinct().select('song', F.monotonically_increasing_id().alias('new_song'))

# get total unique users and songs
unique_users = user_change.count()
unique_songs = song_change.count()
print('Number of unique users: {0}'.format(unique_users))
print('Number of unique songs: {0}'.format(unique_songs))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of unique users: 1019318
Number of unique songs: 384546

In [11]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
# Run string indexer on serveral column
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(dfTaste) for column in ["user", "song"]]
pipeline = Pipeline(stages=indexers)
dfTaste_idx = pipeline.fit(dfTaste).transform(dfTaste)

dfTaste_idx.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+---------+----------+----------+
|                user|              song|playCount|user_index|song_index|
+--------------------+------------------+---------+----------+----------+
|b80344d063b5ccb32...|SOAKIMP12A8C130995|        1|  103159.0|    2979.0|
|b80344d063b5ccb32...|SOAPDEY12A81C210A9|        1|  103159.0|   12804.0|
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|        2|  103159.0|    3509.0|
|b80344d063b5ccb32...|SOBFNSP12AF72A0E22|        1|  103159.0|   31957.0|
|b80344d063b5ccb32...|SOBFOVM12A58A7D494|        1|  103159.0|   20788.0|
+--------------------+------------------+---------+----------+----------+
only showing top 5 rows

In [12]:
# Cast index column to Integer
dfTaste_idx = dfTaste_idx.withColumn("user_index", dfTaste_idx['user_index'].cast(IntegerType()))
dfTaste_idx = dfTaste_idx.withColumn("song_index", dfTaste_idx['song_index'].cast(IntegerType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# cache
tasteDf_with_songId = dfTaste_idx
tasteDf_with_songId.cache()
tasteDf_with_songId.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+---------+----------+----------+
|                user|              song|playCount|user_index|song_index|
+--------------------+------------------+---------+----------+----------+
|b80344d063b5ccb32...|SOAKIMP12A8C130995|        1|    103159|      2979|
|b80344d063b5ccb32...|SOAPDEY12A81C210A9|        1|    103159|     12804|
|b80344d063b5ccb32...|SOBBMDR12A8C13253B|        2|    103159|      3509|
|b80344d063b5ccb32...|SOBFNSP12AF72A0E22|        1|    103159|     31957|
|b80344d063b5ccb32...|SOBFOVM12A58A7D494|        1|    103159|     20788|
+--------------------+------------------+---------+----------+----------+
only showing top 5 rows

In [14]:
# We'll hold out 60% for training, 20% of our data for validation, and leave 20% for testing
seed = 0
(split_60_df, split_a_20_df, split_b_20_df) = tasteDf_with_songId.randomSplit([0.6, 0.2, 0.2], seed = seed)

# Let's cache these datasets for performance
training_df = split_60_df.cache()
validation_df = split_a_20_df.cache()
test_df = split_b_20_df.cache()

print('Training: {0}, validation: {1}, test: {2}\n'.format(
  training_df.count(), validation_df.count(), test_df.count())
)
training_df.show(3)
validation_df.show(3)
test_df.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training: 29020730, validation: 9674561, test: 9678295

+--------------------+------------------+---------+----------+----------+
|                user|              song|playCount|user_index|song_index|
+--------------------+------------------+---------+----------+----------+
|00003a4459f33b929...|SOCEQVU12A6D4F78B3|        3|    753452|     22550|
|00003a4459f33b929...|SOEOODW12AB017A9FB|        1|    753452|    131979|
|00003a4459f33b929...|SOMZHIH12A8AE45D00|        3|    753452|      5527|
+--------------------+------------------+---------+----------+----------+
only showing top 3 rows

+--------------------+------------------+---------+----------+----------+
|                user|              song|playCount|user_index|song_index|
+--------------------+------------------+---------+----------+----------+
|00003a4459f33b929...|SOJTVUM12AB018CB60|        1|    753452|    108742|
|00003a4459f33b929...|SOMZIQI12AB017F9B8|        1|    753452|    112271|
|00005c6177188f12f...|SOGNVXA12

In [15]:
from pyspark.sql.types import DoubleType

#Number of plays needs to be double type, not integers
validation_df = validation_df.withColumn("playCount", validation_df["playCount"].cast(DoubleType()))
validation_df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+---------+----------+----------+
|                user|              song|playCount|user_index|song_index|
+--------------------+------------------+---------+----------+----------+
|00003a4459f33b929...|SOJTVUM12AB018CB60|      1.0|    753452|    108742|
|00003a4459f33b929...|SOMZIQI12AB017F9B8|      1.0|    753452|    112271|
|00005c6177188f12f...|SOGNVXA12A8C14373F|      1.0|    772245|      7240|
|00005c6177188f12f...|SORGGRV12A8AE477C0|      2.0|    772245|      6714|
|00030033e3a2f904a...|SOBEAKR12A6701F536|      3.0|    756373|     20130|
|00030033e3a2f904a...|SOWCEQC12AF72A2B64|      2.0|    756373|     16308|
|00030033e3a2f904a...|SOXJFKD12A3F1EA31A|      1.0|    756373|     11332|
|00030033e3a2f904a...|SOXNYUA12AB017B49B|      3.0|    756373|       767|
|00030033e3a2f904a...|SOXQWUS12A58A7B236|      1.0|    756373|      1187|
|0007c0e74728ca9ef...|SODMFNK12AF72A2A27|      1.0|    861308|     10393|
+--------------------+----------------

In [16]:
# Alternating least squares
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Let's initialize our ALS learner
als = ALS()

# Now set the parameters for the method
als.setMaxIter(3)\
   .setSeed(seed)\
   .setItemCol("song_index")\
   .setRatingCol("playCount")\
   .setUserCol("user_index")

# Create regression evaluator
reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="playCount", metricName="rmse")

# Hyperparameter tuning to find best rank and reg param
tolerance = 0.03
ranks = [4, 8, 12]
regParams = [0.15, 0.2]
errors = [[0]*len(ranks)]*len(regParams)
models = [[0]*len(ranks)]*len(regParams)
err = 0
min_error = float('inf')
best_rank = -1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
i = 0
for regParam in regParams:
    j = 0
    for rank in ranks:
        
        # Set the rank here:
        als.setParams(rank = rank, regParam = regParam)
        # Create the model with these parameters.
        model = als.fit(training_df)
        # Run the model to create a prediction. Predict against the validation_df.
        predict_df = model.transform(validation_df)
        # Remove NaN values from prediction (due to SPARK-14489)
        predicted_plays_df = predict_df.filter(predict_df.prediction != float('nan'))
        predicted_plays_df = predicted_plays_df.withColumn("prediction", F.abs(F.round(predicted_plays_df["prediction"],0)))
        # Run the previously created RMSE evaluator, reg_eval, on the predicted_ratings_df DataFrame
        error = reg_eval.evaluate(predicted_plays_df)
        errors[i][j] = error
        models[i][j] = model
        print ('For rank %s, regularization parameter %s the RMSE is %s' % (rank, regParam, error))
        if error < min_error:
            min_error = error
            best_params = [i,j]
        j += 1
    i += 1

als.setRegParam(regParams[best_params[0]])
als.setRank(ranks[best_params[1]])
# print ('The best model was trained with regularization parameter %s' % regParams[best_params[0]])
# print ('The best model was trained with rank %s' % ranks[best_params[1]])
my_model = models[best_params[0]][best_params[1]]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

For rank 4, regularization parameter 0.15 the RMSE is 7.600068841594392
For rank 8, regularization parameter 0.15 the RMSE is 7.4837036338836445
For rank 12, regularization parameter 0.15 the RMSE is 7.318356017861835
For rank 4, regularization parameter 0.2 the RMSE is 7.4694363719117
For rank 8, regularization parameter 0.2 the RMSE is 7.338156714784233
For rank 12, regularization parameter 0.2 the RMSE is 7.2012332572684965

In [18]:
predicted_plays_df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+---------+----------+----------+----------+
|                user|              song|playCount|user_index|song_index|prediction|
+--------------------+------------------+---------+----------+----------+----------+
|7e2a2bbe97568dfce...|SOUVTSM12AC468F6A7|      1.0|       334|        12|       1.0|
|3ffb4ea1467ad4549...|SOUVTSM12AC468F6A7|      2.0|       977|        12|       1.0|
|e4b426ac0d1cb8ec2...|SOUVTSM12AC468F6A7|      7.0|      1760|        12|       1.0|
|285f4d3965daa1275...|SOUVTSM12AC468F6A7|      3.0|      2615|        12|       1.0|
|2fff793ffc36da7fc...|SOUVTSM12AC468F6A7|      6.0|      2858|        12|       2.0|
|cb650779e089ac8d3...|SOUVTSM12AC468F6A7|      2.0|      4358|        12|       1.0|
|583be18f53885bd02...|SOUVTSM12AC468F6A7|      1.0|      4456|        12|       2.0|
|620983c7b1b25bb6b...|SOUVTSM12AC468F6A7|      3.0|      6812|        12|       1.0|
|573bef8f7c4d79269...|SOUVTSM12AC468F6A7|      1.0|      8099|   

In [19]:
# Testing the model
# In ML Pipelines, this next step has a bug that produces unwanted NaN values. We
# have to filter them out. See https://issues.apache.org/jira/browse/SPARK-14489

test_df = test_df.withColumn("playCount", test_df["playCount"].cast(DoubleType()))
predict_df = my_model.transform(test_df)

# Remove NaN values from prediction (due to SPARK-14489)
predicted_test_df = predict_df.filter(predict_df.prediction != float('nan'))

# Round floats to whole numbers
predicted_test_df = predicted_test_df.withColumn("prediction", F.abs(F.round(predicted_test_df["prediction"],0)))
# Run the previously created RMSE evaluator, reg_eval, on the predicted_test_df DataFrame
test_RMSE = reg_eval.evaluate(predicted_test_df)

print('The model had a RMSE on the test set of {0}'.format(test_RMSE))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The model had a RMSE on the test set of 6.2977267068362535

In [20]:
# Comparing the model with the error from a test set where every rating is the averge number of plays from the training set
avg_plays_df = training_df.groupBy().avg('playCount').select(F.round('avg(playCount)'))

avg_plays_df.show(3)
# Extract the average rating value. (This is row 0, column 0.)
training_avg_plays = avg_plays_df.collect()[0][0]

print('The average number of plays in the dataset is {0}'.format(training_avg_plays))

# Add a column with the average rating
test_for_avg_df = test_df.withColumn('prediction', F.lit(training_avg_plays))

# Run the previously created RMSE evaluator, reg_eval, on the test_for_avg_df DataFrame
test_avg_RMSE = reg_eval.evaluate(test_for_avg_df)

print("The RMSE on the average set is {0}".format(test_avg_RMSE))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------+
|round(avg(playCount), 0)|
+------------------------+
|                     3.0|
+------------------------+

The average number of plays in the dataset is 3.0
The RMSE on the average set is 6.175594688452001